#### If come back to this notebook or have to restart the notebook, start from Check Point. The preprocessed data has been saved to disk.

In [ ]:
%matplotlib inline
import cv2
import os
import random
import numpy as np
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

## Preparing the Data

In [ ]:
TRAIN_DIR = 'input/train/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

In [ ]:
# Explore the dataset
image_id = 10000

# Don't modify below
image_example = cv2.imread(train_images[image_id])
print("Stats of Dog_vs_Cat Training Dataset")
print("Total Images: {}".format(len(train_images)))
print("Dog:          {}".format(len([i for i in train_images if 'dog' in i ])))
print("Cat:          {}".format(len([i for i in train_images if 'cat' in i ])))
print("")
print("Example of Image {}".format(image_id))
print("Image - Shape: {}".format(image_example.shape))
print("Image - Min Value: {}, Max Value: {}".format(image_example.min(), image_example.max()))
print("Label - {}".format('dog' if 'dog' in train_images[image_id] else 'cat'))
display(Image.open(train_images[image_id]))

In [ ]:
#Randomize Data
random.shuffle(train_images)

In [ ]:
# Resize images
ROWS = 64 #Resized image row size
COLS = 64 #Resized image columns size
CHANNELS = 3 #RGB channels

# Don't modify below
feature = []
for i, img in enumerate(train_images): 
    img = cv2.imread(img)
    img = cv2.resize(img, dsize=(ROWS, COLS))
    feature.append(img)
feature = np.array(feature, dtype=np.uint8)

In [ ]:
# Explore the resized image
image_example_new = 10

#Don't modify below
print('Resized Image')
display(Image.fromarray(feature[image_example_new]))
print('')
print('Original Image')
display(Image.open(train_images[image_example_new]))

In [ ]:
# Normalize features
feature = (feature-np.min(feature))/(np.max(feature)-np.min(feature))

In [ ]:
#One-hot encoding on the label
label = [[0, 1] if 'cat' in i else [1, 0] for i in train_images]
label = np.array(label, dtype=np.uint8)

In [ ]:
import pickle
with open('preprocess_image.p', 'wb') as f:
    pickle.dump((feature,label), f)

## Check Point

If come back to this notebook or have to restart the notebook, start from here. The preprocessed data has been saved to disk.

In [80]:
import pickle
import numpy as np
import tensorflow as tf

In [2]:
with open('preprocess_image.p', 'rb') as f:
    feature, label = pickle.load(f)

In [3]:
print("Image - Shape: {}".format(feature.shape[1:]))

Image - Shape: (64, 64, 3)


In [90]:
# Train test split to generate training set and validation set
from sklearn.model_selection import train_test_split
training_size = 0.96 # 90% of the data will be used as training, 
                    #and the remaining 10% will be used as validation set
train_feature, val_feature, train_label, val_label=train_test_split(feature, label, train_size=training_size, random_state=42)

C:\Users\Zaoli\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [92]:
def conv2d(x_tensor, conv_num_outputs, conv_ksize, conv_strides, conv_padding='SAME'):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    :param conv_padding: 'SAME' or 'VALID'
    :param pool_padding: 'SAME' or 'VALID
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    weight =  tf.Variable(tf.random_normal(conv_ksize+(x_tensor.shape[-1].value,)+(conv_num_outputs,),
                                           stddev=0.05))
    bias = tf.Variable(tf.zeros([conv_num_outputs]))
    x = tf.nn.conv2d(x_tensor, weight, strides=(1,)+conv_strides+(1,), padding=conv_padding)
    x = tf.nn.bias_add(x, bias)
    x = tf.nn.relu(x)

    return x 

In [93]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides, conv_padding='SAME', pool_padding='SAME'):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    :param conv_padding: 'SAME' or 'VALID'
    :param pool_padding: 'SAME' or 'VALID
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    weight =  tf.Variable(tf.random_normal(conv_ksize+(x_tensor.shape[-1].value,)+(conv_num_outputs,),
                                           stddev=0.05))
    bias = tf.Variable(tf.zeros([conv_num_outputs]))
    x = tf.nn.conv2d(x_tensor, weight, strides=(1,)+conv_strides+(1,), padding=conv_padding)
    x = tf.nn.bias_add(x, bias)
    x = tf.nn.relu(x)
    x = tf.nn.max_pool(x, ksize=((1,)+ pool_ksize + (1,)),
                      strides=((1,)+ pool_strides + (1,)),
                      padding=pool_padding)
    return x 

In [94]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    flatten_x = tf.reshape(x_tensor, 
                           shape=[tf.shape(x_tensor)[0], x_tensor.shape[1].value*x_tensor.shape[2].value*x_tensor.shape[3].value])
    return flatten_x

In [95]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    weight = tf.Variable(tf.truncated_normal((x_tensor.shape[-1].value, num_outputs),
                                            stddev = 0.05))
    bias = tf.Variable(tf.zeros([num_outputs]))
    fc_x = tf.add(tf.matmul(x_tensor, weight), bias)
    fc_x = tf.nn.relu(fc_x)
    
    return fc_x

In [96]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    weight = tf.Variable(tf.truncated_normal([x_tensor.shape[-1].value,num_outputs],
                                            stddev=0.05))
    bias = tf.Variable(tf.zeros([num_outputs]))
    return tf.add(tf.matmul(x_tensor, weight), bias)

In [97]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides, conv_padding, pool_padding)
    conv = conv2d_maxpool(x, 96, (5, 5), (4, 4), (3, 3), (2, 2), conv_padding='VALID', pool_padding='VALID')
    conv = tf.nn.dropout(conv, keep_prob)
    conv = conv2d_maxpool(conv, 256, (5, 5), (1, 1), (3, 3), (2, 2), conv_padding='SAME', pool_padding='VALID')
    conv = tf.nn.dropout(conv, keep_prob)
    conv = conv2d(conv, 384, (3, 3), (1, 1), conv_padding='SAME')
    conv = tf.nn.dropout(conv, keep_prob)
    conv = conv2d(conv, 384, (3, 3), (1, 1), conv_padding='SAME')
    conv = tf.nn.dropout(conv, keep_prob)
    conv = conv2d_maxpool(conv, 256, (3, 3), (1, 1), (3, 3), (2, 2), conv_padding='SAME', pool_padding='VALID')
    conv = tf.nn.dropout(conv, keep_prob)
    # Function Definition from Above:
    #   flatten(x_tensor)
    flt = flatten(conv)
    ft = tf.nn.dropout(flt, keep_prob)

    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    fc = fully_conn(flt, 4096)
    fc = tf.nn.dropout(fc, keep_prob)
    fc = fully_conn(flt, 4096)
    fc = tf.nn.dropout(fc, keep_prob)

    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    
    # TODO: return output
    return output(fc, 2)

In [98]:
##############################
## Build the Neural Network ##
##############################

# Remove previous weights, bias, inputs, etc..
import tensorflow as tf

tf.reset_default_graph()

# Inputs
ROWS = feature.shape[1]
COLS = feature.shape[2] #Resized image columns size
CHANNELS = feature.shape[3] #RGB channels
image_shape = (ROWS, COLS, CHANNELS)
n_classes = label.shape[1]
x = tf.placeholder(tf.float32, shape = (None, image_shape[0], image_shape[1], image_shape[2]), name='x')
y = tf.placeholder(tf.float32, shape = (None, n_classes), name='y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name='cost')
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [114]:
# Tune Parameters
epochs = 500
batch_size = 4096
val_batch_size = 128
keep_probability = 0.7

In [115]:
basave_model_path = './dog_vs_cat'

print('Training...')
with tf.Session() as sess:
    #Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        train_batch_loss_total=0
        train_batch_acc_total=0 
        train_batch_count=0
        for batch_start in range(0, train_feature.shape[0], batch_size):
            batch_end = min(batch_start + batch_size, train_feature.shape[0])
            feature_batch = train_feature[batch_start: batch_end]
            label_batch = train_label[batch_start: batch_end]

            sess.run(optimizer, feed_dict={
                    x: feature_batch,
                    y: label_batch,
                    keep_prob: keep_probability
                })
            
            train_batch_loss_total += sess.run(cost, feed_dict={
                x: feature_batch,
                y: label_batch,
                keep_prob: 1.0
                })
            train_batch_acc_total += sess.run(accuracy, feed_dict={
                x:feature_batch,
                y:label_batch,
                keep_prob: 1.0
            })
            train_batch_count += 1
        train_loss=train_batch_loss_total/train_batch_count
        train_acc=train_batch_acc_total/train_batch_count
    
    
        val_batch_loss_total = 0
        val_batch_acc_total = 0
        val_batch_count = 0
    
        for val_batch_start in range(0, val_feature.shape[0], val_batch_size):
            val_batch_end = min(val_batch_start + val_batch_size, val_feature.shape[0])
            val_feature_batch = val_feature[val_batch_start:val_batch_end]
            val_label_batch = val_label[val_batch_start:val_batch_end]
        
            val_batch_loss_total += sess.run(cost, feed_dict={
                x: val_feature_batch,
                y: val_label_batch,
                keep_prob: 1.
            })
        
            val_batch_acc_total += sess.run(accuracy, feed_dict={
                x: val_feature_batch,
                y: val_label_batch,
                keep_prob: 1.
            })
            val_batch_count += 1
      
    
        val_loss = val_batch_loss_total/val_batch_count
        val_acc = val_batch_acc_total/val_batch_count
        print('Epoch {:>2}, Training Loss: {:>3.5f}, Training Accuracy: {:.2f}%, Validation Loss: {:>3.5f}, Validation Accuracy: {:.2f}%'.format(epoch+1, train_loss, train_acc*100, val_loss, val_acc*100))
    # Save model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Training...
Epoch  1, Training Loss: 9.70682, Training Accuracy: 50.00%, Validation Loss: 0.69536, Validation Accuracy: 50.86%
Epoch  2, Training Loss: 0.73384, Training Accuracy: 49.96%, Validation Loss: 0.71546, Validation Accuracy: 50.86%
Epoch  3, Training Loss: 0.70098, Training Accuracy: 49.96%, Validation Loss: 0.69433, Validation Accuracy: 50.86%
Epoch  4, Training Loss: 0.69342, Training Accuracy: 49.66%, Validation Loss: 0.69324, Validation Accuracy: 49.14%
Epoch  5, Training Loss: 0.69369, Training Accuracy: 50.04%, Validation Loss: 0.69461, Validation Accuracy: 49.14%
Epoch  6, Training Loss: 0.69378, Training Accuracy: 50.04%, Validation Loss: 0.69393, Validation Accuracy: 49.14%
Epoch  7, Training Loss: 0.69326, Training Accuracy: 50.04%, Validation Loss: 0.69342, Validation Accuracy: 49.14%
Epoch  8, Training Loss: 0.69311, Training Accuracy: 50.04%, Validation Loss: 0.69317, Validation Accuracy: 49.14%
Epoch  9, Training Loss: 0.69297, Training Accuracy: 50.04%, Validat

Epoch 73, Training Loss: 0.36892, Training Accuracy: 83.53%, Validation Loss: 0.47850, Validation Accuracy: 76.01%
Epoch 74, Training Loss: 0.36336, Training Accuracy: 83.60%, Validation Loss: 0.48056, Validation Accuracy: 76.40%
Epoch 75, Training Loss: 0.35683, Training Accuracy: 84.16%, Validation Loss: 0.49023, Validation Accuracy: 76.49%
Epoch 76, Training Loss: 0.34786, Training Accuracy: 85.03%, Validation Loss: 0.46659, Validation Accuracy: 76.93%
Epoch 77, Training Loss: 0.34985, Training Accuracy: 84.82%, Validation Loss: 0.45924, Validation Accuracy: 78.05%
Epoch 78, Training Loss: 0.34529, Training Accuracy: 85.25%, Validation Loss: 0.46186, Validation Accuracy: 78.39%
Epoch 79, Training Loss: 0.33075, Training Accuracy: 85.78%, Validation Loss: 0.46709, Validation Accuracy: 77.40%
Epoch 80, Training Loss: 0.33836, Training Accuracy: 85.10%, Validation Loss: 0.49437, Validation Accuracy: 76.19%
Epoch 81, Training Loss: 0.32542, Training Accuracy: 86.10%, Validation Loss: 0.

KeyboardInterrupt: 